In [118]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [125]:
df = pd.read_csv('train_case2.csv', sep=';')

In [126]:
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


###  Отмасштабирую численные признаки для обучения линейного классификатора

In [127]:
cols_to_transform = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol']

In [128]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.column]
    
logreg = Pipeline([
    ('cols_selector', FeatureSelector(cols_to_transform)),
    ('scaler', MinMaxScaler()),
    ('classifier', LogisticRegression()),
])

In [129]:
y = df['cardio']
X = df.drop('cardio', axis=1)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [131]:
cross_val_score(logreg, X, y, cv=5, scoring='f1')

array([0.63043958, 0.64027334, 0.63358779, 0.63620931, 0.63388783])

In [122]:
result = pd.DataFrame(columns=['model', 'f1', 'precision', 'recall', 'roc_auc'])
def get_metrics(model, model_name, result):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    metrics = {}
    for metric, func in zip(['f1', 'precision', 'recall', 'roc_auc'], 
                            [f1_score, precision_score, recall_score, roc_auc_score]):
        metrics[metric] = func(y_test, pred)
    metrics['model'] = model_name
    result = result.append(metrics, ignore_index=True)
    return result

In [123]:
random_forest = RandomForestClassifier()
catboost = CatBoostClassifier(silent=True, cat_features=['gluc', 'smoke', 'alco', 'active'])
xgboost = XGBClassifier()
for name, classifier in zip(['logreg', 'random_forest', 'xgboost', 'catboost'], 
                            [logreg, random_forest, xgboost, catboost]):
    result = get_metrics(classifier, name, result=result)

In [124]:
result

,model,f1,precision,recall,roc_auc
0,logreg,0.627907,0.628918,0.626899,0.625127
1,random_forest,0.715485,0.736316,0.695800,0.721038
2,xgboost,0.720867,0.750664,0.693346,0.729422
3,catboost,0.720358,0.752338,0.690986,0.729683


### Видно, что все ансамбли справляются с задачей значительно лучше линейной модели

### 5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

#### Лучше будет первая модель, поскольку она меньше ошибается при предсказании первого класса, т.е. у нее выше Precision, это отразится на метрике PR_AUC, которая будет высокой. Вторая же модель часто ошибается и выдает неверные предсказания для объектов 0 класса, классифицируя их как 1 класс. Мы увидим это только на метрике PR_AUC